### Imports
* Note that module level tolerances are printed on module import. 
  * These can be overriden after import by just setting them eg. `mseutils.MZ_PPM = 7`


In [ ]:
import mseutils
import os #just needed for loading datafiles

### Get Data Files from Module Path

In [ ]:
modpath = mseutils.__path__[0]
data_path = os.path.join(modpath,'..','examples','data')
rep_file = os.path.join(data_path,"SW218968-1_replicated.csv")
rep_frag_file = os.path.join(data_path,"SW218968-1_rep_fragments.csv")
mgf_file = os.path.join(data_path,"20160928_SW218968-1_iDTs_CPPIS.mgf")


### Load Replicated CSV files
The `load_rep_and_frags_csv` func from the `file_parsers` reads in a replicated parent mass csv and the accompanying fragments file. The resulting `MseSpec`'s are then combined based on retetion time, drift time (ccs), and parent mz. Their fragments are added and averaged (m/z and intensity).

In [ ]:
rep_mse_specs = mseutils.file_parsers.load_rep_and_frags_csv(rep_file,rep_frag_file)

### Load a MGF file
You can also load a mgf file directly by using the `load_mgf` function in `file_parsers`.

In [ ]:
mgf_mse_specs = mseutils.file_parsers.load_mgf(mgf_file)

## MseSpec Objects
The `MseSpec` object is pretty powerful class that holds information about an MSe spectrum as the name might suggest. 

This provides access to :
* parent mz (mse.mz)
* ccs (mse.mz)
* retention time (in min) (mse.rt)
* ms2_data (mse.ms2_data)
* the number of spectra that were combined to make it up (mse.n)
* and their mgf filenames (mse.mgf_files)
* and others...
    * like a ms2 fingerprint (mse.ms2vect) made by binning (1 amu) ms2_data

In [ ]:
mse = rep_mse_specs[0]
print("m/z: {}".format(mse.mz))
print("mse: {}".format(mse.ccs))
print("rt: {}".format(mse.rt))
print("ms2 data: {}".format(mse.ms2_data))

There is also a IPython/Jupyter Notebook hook for visualizing the ms2_data. The coloring is the matplotlib "viridis" color scheme and represents relative intensity of each fragment. 

In [ ]:
mse.ms2_data

### Source Fragmentation
To look at source fragments `src_frags` will compare each fragment within the retention time error window and check if the parent mz is in the ms2_data of another spec in that range. The fuction then adds the source frags to the `mse.src_frags` list. In the future I'll make the ms3 trees...


In [ ]:
combined_src_frags = mseutils.src_frags(rep_mse_specs)

In [ ]:
specs_wsrc_frags = [mse for mse in combined_src_frags if len(mse.src_frags)>0]
print("Number of Specs with source fragments: {}".format(len(specs_wsrc_frags)))
specs_wsrc_frags[0].ms2_data

In [ ]:
specs_wsrc_frags[0].src_frags[0]

I think this spec with source fragmentation in this file is noise, the copius number of fragments with small mass deltas of the above doesn't ring true to me... TBD

### Misc and etc

There's a bunch of smaller classes used for comparing mz's rt's and ccs's. They all incorporate some basic arithmatic and comparison operators. Below is an example with `mseutils.MZ` objects

In [ ]:
mz1 = mseutils.MZ(123.4678,z=1)
mz2 = mseutils.MZ(123.4678001,z=1)
mz3 = mseutils.MZ(123.4600,z=1)
mz4 = mseutils.MZ(123.4678,z=2)
mz1 == mz2

In [ ]:
mz1 == mz3

In [ ]:
mz1==mz4

In [ ]:
mz1 + mz3 

In [ ]:
mz1 > mz3